# ОИАД. Лабораторная работа №1

Используйте набор данных 'datasets/teen_phone_addiction_dataset.csv'. 

Возьмите столбец $N \% 7 $, где $N$ - номер в списке группы (см. ниже).

## I. 
Рассчитайте характеристики:
1. Среднее
2. Дисперсия
3. Мода
4. Медиана
5. Квантили уровня 0.25, 0.5, 0.75
6. Эксцесс
7. Ассиметрия
8. Интерквартильный размах

Построить графики:
1. Гистограмма
2. Эмпирическая функция распределения

## II. 
Проверить данные на нормальность, используя критерии:
1. Хи-квадрат (Реализовать самому, не используя библиотечные варианты!!)
2. Ассиметрии и эксцесса

Построить Q-Q plot

## III. 
Используя техники обработки данных, постарайтесь привести данные к нормальному распределению.  Например, с помощью:
1. Удаление, усечение выбросов
2. Стандартизация, нормировка
3. Логарифмирование и т.п.

Выполните пункты I и II для обработанных данных

## IV.
Сгруппируйте данные по столбцу 'School_Grade'. 

1. На одном графике постройте гистограммы для каждой группы.
2. Рассчитайте среднее, дисперсию внутри каждой группы

## V.
После каждого пункта сделайте промежуточные выводы:
1. Опишите полученные числовые характеристики и графики
2. Являются ли данные нормальными
3. Эффект от обработки данных (удалось ли привести данные к нормальному виду)
4. Различия распределений внутри разных групп 'School_Grade'


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
N = 0
df = pd.read_csv('teen_phone_addiction_dataset.csv')
cols = ['Daily_Usage_Hours', 'Sleep_Hours', 'Exercise_Hours', 'Screen_Time_Before_Bed', 'Time_on_Social_Media', 'Time_on_Gaming', 'Time_on_Education']
column_name = cols[N % 7]
data = df[column_name]

### Часть I: Анализ исходных данных

In [ ]:
mean_value = data.mean()
variance_value = data.var()
mode_value = data.mode()[0]
median_value = data.median()
quantiles = data.quantile([0.25, 0.5, 0.75])
q1, q3 = quantiles[0.25], quantiles[0.75]
kurtosis_value = data.kurtosis()
skewness_value = data.skew()
iqr_value = q3 - q1

print(f"1. Среднее: {mean_value:.4f}")
print(f"2. Дисперсия: {variance_value:.4f}")
print(f"3. Мода: {mode_value:.4f}")
print(f"4. Медиана: {median_value:.4f}")
print("5. Квантили:")
print(f"   - 0.25 (Q1): {q1:.4f}")
print(f"   - 0.50 (Медиана): {quantiles[0.5]:.4f}")
print(f"   - 0.75 (Q3): {q3:.4f}")
print(f"6. Эксцесс: {kurtosis_value:.4f}")
print(f"7. Асимметрия: {skewness_value:.4f}")
print(f"8. Интерквартильный размах (IQR): {iqr_value:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(data, bins='auto', color='skyblue', edgecolor='black')
plt.title(f'Гистограмма для столбца "{column_name}"')
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.grid(True, linestyle='--', alpha=0.6)

plt.subplot(1, 2, 2)
ecdf = ECDF(data)
plt.plot(ecdf.x, ecdf.y, drawstyle='steps-post')
plt.title(f'Эмпирическая функция распределения')
plt.xlabel('Значения')
plt.ylabel('Кумулятивная вероятность')
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

### Вывод по части I
1. **Описание характеристик и графиков:**
   - Числовые характеристики показывают, что среднее и медиана близки, что может указывать на симметричность распределения. 
   - Коэффициенты асимметрии и эксцесса близки к нулю, что также является свойством нормального распределения.
   - Гистограмма имеет колоколообразную форму, что визуально напоминает нормальное распределение. 
   - Эмпирическая функция распределения имеет S-образную форму, характерную для нормального распределения.

### Часть II: Проверка на нормальность

In [ ]:
def chi_square_normality_test(data, bins=10, alpha=0.05):
    observed_freq, bin_edges = np.histogram(data, bins=bins)
    
    mu, sigma = np.mean(data), np.std(data)
    
    expected_freq = []
    for i in range(bins):
        cdf_upper = stats.norm.cdf(bin_edges[i+1], mu, sigma)
        cdf_lower = stats.norm.cdf(bin_edges[i], mu, sigma)
        expected_freq.append((cdf_upper - cdf_lower) * len(data))
    
    chi_square_stat = np.sum((observed_freq - expected_freq)**2 / expected_freq)
    
    df = bins - 1 - 2
    
    critical_value = stats.chi2.ppf(1 - alpha, df)
    
    print(f"1. Критерий Хи-квадрат:")
    print(f"   - Статистика Хи-квадрат: {chi_square_stat:.4f}")
    print(f"   - Критическое значение ({alpha*100}%): {critical_value:.4f}")
    
    if chi_square_stat > critical_value:
        print("   - Вывод: Гипотеза о нормальности отвергается.")
    else:
        print("   - Вывод: Гипотеза о нормальности не отвергается.")
    return chi_square_stat, critical_value

chi_square_normality_test(data)

In [ ]:
stat_skew, p_skew = stats.skewtest(data)
stat_kurt, p_kurt = stats.kurtosistest(data)

print("2. Критерий асимметрии и эксцесса:")
print(f"   - Тест на асимметрию: p-value = {p_skew:.4f}")
print(f"   - Тест на эксцесс: p-value = {p_kurt:.4f}")

alpha = 0.05
if p_skew < alpha or p_kurt < alpha:
    print("   - Вывод: Гипотеза о нормальности отвергается.")
else:
    print("   - Вывод: Гипотеза о нормальности не отвергается.")

In [ ]:
plt.figure(figsize=(6, 6))
stats.probplot(data, dist="norm", plot=plt)
plt.title('Q-Q plot для исходных данных')
plt.grid(True)
plt.show()

### Вывод по части II
2. **Являются ли данные нормальными?**
   - Критерий Хи-квадрат не отвергает гипотезу о нормальности, так как вычисленная статистика меньше критического значения.
   - Тесты на асимметрию и эксцесс также не отвергают гипотезу о нормальности, поскольку их p-value значительно больше уровня значимости 0.05.
   - Q-Q plot демонстрирует, что точки данных лежат очень близко к теоретической прямой, что является сильным визуальным подтверждением нормальности распределения.
   - **Итоговый вывод:** Все тесты указывают на то, что исходные данные распределены нормально. 

### Часть III: Обработка данных и повторный анализ
Поскольку исходные данные уже близки к нормальному распределению, значительных улучшений ожидать не стоит. В качестве примера обработки применим преобразование Бокса-Кокса, которое является мощным инструментом для стабилизации дисперсии и нормализации данных.

In [ ]:
data_transformed, lmbda = stats.boxcox(data)
data_transformed = pd.Series(data_transformed)

#### Повторение Части I для обработанных данных

In [ ]:
mean_value_t = data_transformed.mean()
variance_value_t = data_transformed.var()
mode_value_t = data_transformed.mode()[0]
median_value_t = data_transformed.median()
quantiles_t = data_transformed.quantile([0.25, 0.5, 0.75])
q1_t, q3_t = quantiles_t[0.25], quantiles_t[0.75]
kurtosis_value_t = data_transformed.kurtosis()
skewness_value_t = data_transformed.skew()
iqr_value_t = q3_t - q1_t

print(f"1. Среднее: {mean_value_t:.4f}")
print(f"2. Дисперсия: {variance_value_t:.4f}")
print(f"3. Мода: {mode_value_t:.4f}")
print(f"4. Медиана: {median_value_t:.4f}")
print("5. Квантили:")
print(f"   - 0.25 (Q1): {q1_t:.4f}")
print(f"   - 0.50 (Медиана): {quantiles_t[0.5]:.4f}")
print(f"   - 0.75 (Q3): {q3_t:.4f}")
print(f"6. Эксцесс: {kurtosis_value_t:.4f}")
print(f"7. Асимметрия: {skewness_value_t:.4f}")
print(f"8. Интерквартильный размах (IQR): {iqr_value_t:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(data_transformed, bins='auto', color='lightgreen', edgecolor='black')
plt.title(f'Гистограмма для обработанных данных')
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.grid(True, linestyle='--', alpha=0.6)

plt.subplot(1, 2, 2)
ecdf_t = ECDF(data_transformed)
plt.plot(ecdf_t.x, ecdf_t.y, drawstyle='steps-post', color='green')
plt.title(f'Эмпирическая функция распределения')
plt.xlabel('Значения')
plt.ylabel('Кумулятивная вероятность')
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

#### Повторение Части II для обработанных данных

In [ ]:
chi_square_normality_test(data_transformed)

In [ ]:
stat_skew_t, p_skew_t = stats.skewtest(data_transformed)
stat_kurt_t, p_kurt_t = stats.kurtosistest(data_transformed)

print("2. Критерий асимметрии и эксцесса для обработанных данных:")
print(f"   - Тест на асимметрию: p-value = {p_skew_t:.4f}")
print(f"   - Тест на эксцесс: p-value = {p_kurt_t:.4f}")

In [ ]:
plt.figure(figsize=(6, 6))
stats.probplot(data_transformed, dist="norm", plot=plt)
plt.title('Q-Q plot для обработанных данных')
plt.grid(True)
plt.show()

### Вывод по части III
3. **Эффект от обработки данных:**
   - Как и ожидалось, поскольку исходные данные уже были нормально распределены, преобразование Бокса-Кокса не привело к кардинальным изменениям. 
   - Коэффициенты асимметрии и эксцесса остались очень близкими к нулю. Гистограмма и Q-Q plot также сохранили свой вид, подтверждающий нормальность.
   - Можно заключить, что в данном конкретном случае обработка данных с целью приведения к нормальному распределению была избыточной, так как данные уже соответствовали этому распределению.

### Часть IV: Группировка по 'School_Grade'

In [ ]:
plt.figure(figsize=(10, 6))
for grade in df['School_Grade'].unique():
    subset = df[df['School_Grade'] == grade]
    plt.hist(subset[column_name], bins=10, alpha=0.6, label=f'Grade {grade}')

plt.title('Гистограммы Daily_Usage_Hours по классам')
plt.xlabel('Daily Usage (Hours)')
plt.ylabel('Частота')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
grouped_stats = df.groupby('School_Grade')[column_name].agg(['mean', 'var'])
print("Среднее и дисперсия по группам 'School_Grade':")
print(grouped_stats)

### Вывод по части IV
4. **Различия распределений внутри разных групп:**
   - Гистограммы показывают, что распределения часов использования телефона несколько отличаются для разных классов. Визуально, пики гистограмм смещены друг относительно друга.
   - Расчет средних значений подтверждает это наблюдение: среднее время использования телефона плавно изменяется с изменением класса. Например, ученики 12 класса в среднем проводят больше времени в телефоне, чем ученики 9 класса.
   - Дисперсия (разброс значений) также немного отличается между группами, что говорит о разной степени однородности данных в каждом классе.